In [48]:
import pandas as pd
import numpy as np

dft = pd.read_csv("correct_times.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("complete_distances.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [49]:
def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)
def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

In [50]:
table=pd.read_csv("route.csv", delimiter=";")
time_table=pd.read_csv("route_time.csv", delimiter=";")

In [51]:
import datetime
def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

In [52]:
gained_time1=0
to_2=[]
days = ['martedi', 'mercoledi','giovedì']
#for day in days:
day='martedi'
for i in range(len(time_table[day].dropna())-2):
    try:
        diff=difference(day, i, i+1)
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][i+1]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time1 :
            gained_time1+=(diff-ctime)
            print(f"{day}: primo : {paese} : {gained_time1}")
        else:
            gained_time1+=diff
            print(f"{day}: failed : {ctime} : {diff} : {gained_time1}")
            to_2.append(i)
    except:
        print(value)
    #if diff<dft[row[paese].values]:
    #    print("Not acceptable!")

martedi: primo : Cremona, Felice Cavallotti, 2 : 54.0
martedi: primo : Casalpusterlengo : 69.0
martedi: primo : Cremona, Ugolani Dati, 4 : 53.0
martedi: primo : Crema, Civerchi : 29.0
martedi: failed : 47.0 : 5.0 : 34.0
martedi: failed : 200.0 : 10.0 : 44.0
martedi: failed : 205.0 : 10.0 : 54.0
martedi: failed : 561.0 : 20.0 : 74.0
martedi: failed : 527.0 : 10.0 : 84.0
martedi: primo : Offanengo : 40.0
martedi: failed : 72.0 : 5.0 : 45.0
martedi: primo : Gussola : 48.0
martedi: primo : Solarolo : 204.0
martedi: primo : Ostiano : 204.0
martedi: primo : Scandolara Ravara, Italia : 205.0
martedi: primo : Persico Dosimo : 174.0
martedi: primo : Casalmaggiore : 140.0
martedi: primo : Gadesco : 119.0
martedi: primo : Viadana : 66.0
martedi: primo : Pescarolo : 32.0


In [53]:
impossible=[]
gained_time2=0
for i in range(len(to_2)-1):
    try:
        curr=to_2[i]
        diff=difference(day, curr, to_2[i+1])
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][to_2[i+1]]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time2 :
            gained_time2+=(diff-ctime)
            print(f"{day}: secondo : {paese} : {gained_time2}")
        else:
            print(f"{day}: impossible : {paese} : {diff}  {ctime}")
            impossible.append(i)
    except:
        print(f"-----{value}")

martedi: impossible : Bonemerse : 5.0  37.0
martedi: impossible : Sospiro : 10.0  193.0
martedi: impossible : Vailate : 10.0  42.0
martedi: impossible : Pianengo : 20.0  521.0
martedi: secondo : Offanengo : 20.0


In [54]:
start = dft[get_row(dft, 'Cremona, sesto 39').values]
day='martedi'
#Liste dei due furgoni
f1 = []
f2 = []
#indici
i1 = -1
i2 = -1
#Le righe delle distanze del corrente paese in cui si trovano i furgoni, inizialmente lo stesso
row1 = start
row2 = start
#Lista dei paesi non raggiungibili
impossibile = []
for i in range(len(time_table[day].dropna())):
    paese=table[day][i]
    #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
    value1=row1.loc[:,get_column(dft, paese).values]
    value2=row2.loc[:,get_column(dft, paese).values]
    #Salvataggio del tempo utile per raggiungere la destinazione in base alla tabella degli orari
    if(i1 != -1 and i2 != -1):
        diff1=difference(day, i1, i)
        diff2=difference(day, i2, i)
    else:
        #Per la prima tratta l'indice non ha valore utile, si ottiene la differenza staticamente
        if(i1 == -1):
            diff1 = value1.values[0][0]
        if(i2 == -1):
            diff2 = value2.values[0][0]
    #Se uno dei due furgoni può rispettare l'orario (con tolleranza 5%) si sceglie il migliore
    if(float(value1.values[0][0])<= float(diff1) + (float(diff1)/20) or float(value2.values[0][0])<= float(diff2) + float(diff2)/20):
        if(value1.values[0][0] <= value2.values[0][0]):
            f1.append(i)
            i1 = i
            row1 = dft[get_row(dft, table[day][i]).values]
        else:
            f2.append(i)
            i2 = i
            row2 = dft[get_row(dft, table[day][i]).values]
    #Impossibilità di raggiungere la locazione in tempo
    else:
        print(f"current time1 {diff1} time {float(value1.values[0][0])} , current time2 {diff2} time {float(value2.values[0][0])}")
        impossibile.append(i)
print(f1)
print(f2)
print(impossibile)

current time1 10.0 time 200.0 , current time2 15.0 time 208.0
current time1 40.0 time 522.0 , current time2 20.0 time 561.0
current time1 15.0 time 20.0 , current time2 5.0 time 72.0
[0, 5, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[1, 2, 3, 4, 7, 10, 21]
[6, 8, 11]


In [45]:
for i in range(len(dftp)):
    for j in range(len(dftp.columns)):
        curr=dftp.iloc[i].iloc[j]
        dftp.iat[i,j]=round(float(curr)/1000)      

In [47]:
dftp.to_csv('correct_times.csv',sep=';')